In [1]:
import sys
sys.path.append('../../../training_data/')

In [3]:
import pickle

list_of_same_grpahs = pickle.load(open('../../../training_data/data/list_of_same_graphs.pkl', 'rb'))
print(f"Number of unique tasks: {len(list_of_dags)}")

Number of unique tasks: 665


In [4]:
import torch
from torch_geometric.data import Data
import sys
# sys.path.append('../')

# string_to_match = '../'
# sys.path = [path for path in sys.path if string_to_match not in path]
# print(sys.path)

# from gcn.dataset import convert_edge_index

def convert_edge_index(edge_index, num_of_tasks):
    converted_edge_index = []
    node_mapping = {'Start': 0, 'Exit': num_of_tasks}

    for src, dest in edge_index:
        if src == 'Start':
            src = node_mapping[src]

        if dest == 'Exit':
            dest = node_mapping[dest]

        converted_edge_index.append((src, dest))

    return converted_edge_index

def convert_data(data_dict):
    task_dag = data_dict['task_dag']
    task_processing_time = float(data_dict['network_processing_time'])
    target_value = torch.tensor([task_processing_time]).float()

    task_graph = task_dag.graph
    edge_index = list(task_graph.edges)
    
    total_tasks = len(task_graph.nodes)
    last_task = len(task_graph.nodes) - 1
    
    converted_edge_index = convert_edge_index(edge_index, last_task)
    converted_edge_index_torch = torch.tensor(converted_edge_index, dtype=torch.int).t().contiguous()

    dummy_input = torch.ones(total_tasks).view(-1,1)
    data = Data(x=dummy_input,edge_index=converted_edge_index_torch, y=target_value)

    return data

def LatNetInference(data_dict):
    model_path = "../gcn/results/application_latency_model/LatNet_500.pth"
    model = torch.load(model_path,  map_location=torch.device('cpu'))
    input = convert_data(data_dict)
    pred_latency = model(input)
    actual_latency = input.y
    return pred_latency.item(), actual_latency.item()


test_data =  list_of_same_graphs[0][0]

LatNetInference(test_data)


NameError: name 'list_of_same_graphs' is not defined